# Dependencies

In [1]:
import argparse
import os
from solver import Solver
from helpers import set_seed
import torchvision.transforms as transforms
import main
from collections import OrderedDict
from helpers import RunBuilder

set_seed(0) # important

%load_ext autoreload
%autoreload 2

# VGG16

In [2]:
vgg16_trained = torchvision.models.vgg16(pretrained=True)
vgg16_untrained = torchvision.models.vgg16()

In [3]:
def modify_model(model, input_channels, output_units):
    '''
    Parameters
    
    model: instance of a pytorch model to be modified
    input_channels: channels of input tensor
    output_units: number of units in the last layer
    '''
    model.features[0] = nn.Conv2d(input_channels, 64, kernel_size=3, stride=1, padding=1, dilation=1, groups=1, bias=True)
    model.classifier[6] = nn.Linear(4096, output_units)

In [4]:
modify_model(vgg16_trained, 2, 512)

Test with a random input tensor:

In [5]:
x = torch.randn(1, 2, 256, 256) # (256, 256, 3)
output = vgg16_trained(x)
print(output.shape)

torch.Size([1, 512])


In [7]:
vgg16_trained

VGG(
  (features): Sequential(
    (0): Conv2d(2, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

Features:

In [8]:
vgg16_trained.features

Sequential(
  (0): Conv2d(2, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (17): Conv2d(256, 512, kernel_si

In [9]:
vgg16_trained.classifier

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=512, bias=True)
)

Do the same with the untrained version:

In [10]:
modify_model(vgg16_untrained, 2, 512)

## RunBuilder() functionality:

In [8]:
params_example = OrderedDict(
            lr=[0.01, 0.03],
            batch_size=[128, 256]
        )

In [9]:
runs = RunBuilder.get_runs(params_example)
runs 

[Run(lr=0.01, batch_size=128),
 Run(lr=0.01, batch_size=256),
 Run(lr=0.03, batch_size=128),
 Run(lr=0.03, batch_size=256)]

In [10]:
run = runs[0]
run

Run(lr=0.01, batch_size=128)

In [13]:
print("lr: {}, batch_size: {}".format(run.lr, run.batch_size))

lr: 0.01, batch_size: 128


In [14]:
for run in runs:
    print(run, run.lr, run.batch_size)

Run(lr=0.01, batch_size=128) 0.01 128
Run(lr=0.01, batch_size=256) 0.01 256
Run(lr=0.03, batch_size=128) 0.03 128
Run(lr=0.03, batch_size=256) 0.03 256


So to calculate different runs it goes like this:

In [16]:
for run in RunBuilder.get_runs(params_example):
    # do stuff 
    pass

# Main.py


Get a basic config:

In [2]:
%run main.py --save_best_model --early_stopping

Or get some help on how to setup your own config:

In [7]:
%run main.py -h

usage: main.py [-h] [--image_size IMAGE_SIZE] [--input_ch INPUT_CH]
               [--output_ch OUTPUT_CH] [--num_epochs NUM_EPOCHS]
               [--num_epochs_decay NUM_EPOCHS_DECAY] [--batch_size BATCH_SIZE]
               [--num_workers NUM_WORKERS] [--lr LR] [--optimizer OPTIMIZER]
               [--beta1 BETA1] [--beta2 BETA2] [--momentum MOMENTUM]
               [--lr_decay LR_DECAY] [--early_stopping] [--save_best_model]
               [--mode MODE] [--model_type MODEL_TYPE]
               [--model_path MODEL_PATH] [--train_path TRAIN_PATH]
               [--valid_path VALID_PATH] [--test_path TEST_PATH]
               [--result_path RESULT_PATH]

optional arguments:
  -h, --help            show this help message and exit
  --image_size IMAGE_SIZE
                        w x h of input image
  --input_ch INPUT_CH   number of channels of input image
  --output_ch OUTPUT_CH
                        number of output nodes
  --num_epochs NUM_EPOCHS
                        number of

Example:

In [6]:
# %run main.py --image_size=10
# config # notice how image_size changed from 224 to 10

Now the config is a global variable:

In [18]:
config

Namespace(batch_size=1, beta1=0.5, beta2=0.999, early_stopping=True, image_size=224, input_ch=2, lr=0.0002, lr_decay=0, mode='train', model_path='./models', model_type='Tester', momentum=0.9, num_epochs=100, num_epochs_decay=70, num_workers=0, optimizer='Adam', output_ch=512, result_path='./results/', save_best_model=True, test_path='./dataset/test/', train_path='./dataset/train/', valid_path='./dataset/valid/')

Pass config and get a basic solver (this will be automated later):

In [3]:
solver = get_solver(config)

Now you can access the solver and its attributes and methods:

In [23]:
print("batch_size: {}, image_size: {}, input_ch: {}".format(solver.batch_size, solver.image_size, solver.input_ch))

batch_size: 1, image_size: 224, input_ch: 2


For simplicity we'll only use a dictionary of fixed parameters for now:

In [30]:
solver.params

OrderedDict([('lr', [0.001, 0.003, 0.01, 0.03]),
             ('batch_size', [128, 256, 512]),
             ('patience', [1])])

Check "patience counter" (with patience=20):

In [4]:
set_seed(0) # always set_seed
solver.train()

,run,epoch,train loss,train accuracy,valid loss,valid accuracy,epoch duration,run duration,patience counter,lr,batch_size
0,1,1,0.657067,0.76412,0.461917,0.8346,11.533120,12.357120,0,0.001,128
1,1,2,0.413867,0.85174,0.396629,0.8598,11.888335,24.480692,0,0.001,128
2,1,3,0.365543,0.86858,0.366897,0.8688,11.495014,36.242698,0,0.001,128
3,1,4,0.334520,0.87772,0.345511,0.8744,11.836003,48.281687,0,0.001,128
4,1,5,0.314219,0.88408,0.344319,0.8743,11.465002,59.995687,0,0.001,128
5,1,6,0.296588,0.89184,0.345007,0.8767,10.825001,71.068686,1,0.001,128
